In [25]:
# -*- coding: utf-8 -*-

import numpy as np
from scipy.optimize import fsolve
import pandas as pd
import re

# =============================================================================
# CLASSE SISTEMA REDE ARTERIAL CIRCULO DE WILLIS
# =============================================================================
class SistemaRedeArterial:
    def __init__(self, P_contorno, geometrias_ramos, topologia, viscosidade = 3.5e-3):

        #ATRIBUTOS DE ENTRADA DA CLASSE PRESSÃO, GEOMETRIA DOS RAMOS, TOPOLOGIA DELES, E A VISCOSIDADE DO SANGUE.
        self.P_contorno = P_contorno
        self.geometrias = geometrias_ramos
        self.topologia = topologia
        self.mu = viscosidade

        # CONSTRUÇÃO DOS NÓS, RAMOS.
        self.nos_internos = sorted(self.topologia['nos_internos'])
        self.vasos = sorted(self.geometrias.keys())
        self.validar_topologia()
        
        # CÁLCULO DAS RESISTÊNCIAS HIDRÁULICAS E CRIAÇÃO DOS DICIONÁRIOS DE PRESSÕES E VAZÃO.
        self.resistencias = {}
        for nome, (L, D) in self.geometrias.items():
            if D <= 0: raise ValueError(f"O diâmetro para '{nome}' deve ser positivo.")
            if L <= 0: raise ValueError(f"O comprimento para '{nome}' deve ser positivo.")
            self.resistencias[nome] = (352 * self.mu * L) / (np.pi * (D**4))
        self.pressoes_calculadas = {}
        self.vazoes_calculadas_m3s = {}
        self.vazoes_calculadas_Ls = {}

        # VALIDAR TOPOLOGIA É UM MÉTODO QUE VAI IDENTIFICAR SE OS RAMOS E NÓS EXISTEM DENTRO DAS CONEXÕES EM TOPOLOGIA
    
    
    def validar_topologia(self):
        nos_definidos = set(self.nos_internos) | set(self.P_contorno.keys())
        for nome_ramo, conexoes in self.topologia['ramos'].items():
            if nome_ramo not in self.geometrias:
                raise ValueError(f"Ramo '{nome_ramo}' da topologia não encontrado em 'geometrias_ramos'.")
            if not all(no in nos_definidos for no in conexoes):
                raise ValueError(f"Nós do ramo '{nome_ramo}' não foram definidos em 'nos_internos' ou 'P_contorno'.")

    def _sistema_equacoes(self, incognitas):
        num_nos_internos = len(self.nos_internos)
        pressoes_incognitas = dict(zip(self.nos_internos, incognitas[:num_nos_internos]))
        vazoes = dict(zip(self.vasos, incognitas[num_nos_internos:]))
        pressoes_completas = {**pressoes_incognitas, **self.P_contorno}
        equacoes = []
        for nome_ramo in self.vasos:
            no_inicio, no_fim = self.topologia['ramos'][nome_ramo]
            P_inicio = pressoes_completas[no_inicio]
            P_fim = pressoes_completas[no_fim]
            R = self.resistencias[nome_ramo]
            Q = vazoes[nome_ramo]
            equacoes.append(P_inicio - P_fim - R * Q)
        for no_interno in self.nos_internos:
            fluxo_total_no_no = 0
            for nome_ramo in self.vasos:
                no_inicio, no_fim = self.topologia['ramos'][nome_ramo]
                if no_fim == no_interno: fluxo_total_no_no += vazoes[nome_ramo]
                if no_inicio == no_interno: fluxo_total_no_no -= vazoes[nome_ramo]
            equacoes.append(fluxo_total_no_no)
        return equacoes

    def resolver(self):
        num_nos_internos = len(self.nos_internos)
        num_ramos = len(self.vasos)
        estimativa_P = np.mean(list(self.P_contorno.values()))
        estimativa_inicial = [estimativa_P] * num_nos_internos + [1e-5] * num_ramos
        try:
            solucao = fsolve(self._sistema_equacoes, estimativa_inicial)
            self.pressoes_calculadas = dict(zip(self.nos_internos, solucao[:num_nos_internos]))
            self.vazoes_calculadas_m3s = dict(zip(self.vasos, solucao[num_nos_internos:]))
            self.vazoes_calculadas_Ls = {n: q*1000 for n, q in self.vazoes_calculadas_m3s.items()}
            return True
        except Exception as e:
            print(f"Erro ao resolver o sistema: {e}")
            return False
            
    # =========================================================================
    # EXIBIÇÃO DOS RESULTADOS
    # =========================================================================
    def exibir_equacoes(self):
        """
        Mostra as equações do sistema em formato simbólico e legível.
        """
        print("\n" + "="*50)
        print("      Equações do Sistema a ser Resolvido")
        print("="*50)


        print("\n--- Equações de Perda de Carga (ΔP = R * Q) ---")
        # Itera sobre a lista ordenada de ramos para garantir consistência
        for nome_ramo in self.vasos:
            no_inicio, no_fim = self.topologia['ramos'][nome_ramo]
            # Formata a equação no formato P_inicio - P_fim = R * Q
            # Ex: Pe - PA = Rramo1 * Qramo1
            print(f"  P{no_inicio} - P{no_fim} = R{nome_ramo} * Q{nome_ramo}")

        # 2. Equações de Conservação de Massa
        print("\n--- Equações de Conservação de Massa (ΣQ = 0) ---")
        # Itera sobre a lista ordenada de nós internos
        for no_interno in self.nos_internos:
            termos_vazao = []
            # Encontra todos os fluxos de entrada e saída para o nó atual
            for nome_ramo in self.vasos:
                no_i, no_f = self.topologia['ramos'][nome_ramo]
                if no_f == no_interno:  # Vazão de entrada
                    termos_vazao.append(f"+Q{nome_ramo}")
                if no_i == no_interno:  # Vazão de saída
                    termos_vazao.append(f"-Q{nome_ramo}")
            
            # Formata a equação no formato Q_in1 + Q_in2 - Q_out1 = 0
            # Ex: Q1 - Q2 - Q5 = 0
            corpo_eq = " ".join(termos_vazao).lstrip('+')
            print(f"  Para o Nó '{no_interno}': {corpo_eq.strip()} = 0")

    def exibir_resultados(self):
        if not self.pressoes_calculadas:
            print("O sistema ainda não foi resolvido ou a resolução falhou.")
            return
        print("\n" + "="*50)
        print("         Resultados da Simulação da Rede Arterial")
        print("="*50)
        print("\n--- Condições de Borda e Parâmetros ---")
        for nome_no, pressao in self.P_contorno.items():
            print(f"Pressão de Borda no Nó '{nome_no}': {pressao:.2f} Pa")
        print(f"Viscosidade do Sangue (μ): {self.mu:.2e} Pa.s")
        
        # --- Alteração para exibir Resistências em ordem crescente ---
        print("\n--- Resistências Hidráulicas Calculadas ---")
        sorted_resistencias_items = sorted(self.resistencias.items(), key=lambda item: self._sort_key_ramo(item[0]))
        for nome_ramo, resistencia in sorted_resistencias_items:
            print(f"   Resistência '{nome_ramo}': {resistencia:.2e} Pa.s/m³")
        
        # --- Alteração para exibir Pressões em ordem crescente de nó ---
        print("\n--- Pressões Nodais Calculadas ---")
        all_nodes_pressure = {**self.P_contorno, **self.pressoes_calculadas}
        sorted_nodes_pressure_items = sorted(all_nodes_pressure.items()) # Ordena por nome do nó (alfabético)
        for nome_no, pressao in sorted_nodes_pressure_items:
            print(f"   Pressão no Nó '{nome_no}': {pressao:.2f} Pa ")
        
        # --- Alteração para exibir Vazões em ordem crescente ---
        print("\n--- Vazões Calculadas nos Ramos ---")
        sorted_vazoes_items = sorted(self.vazoes_calculadas_Ls.items(), key=lambda item: self._sort_key_ramo(item[0]))
        for nome_ramo, q_Ls in sorted_vazoes_items:
            no_inicio, no_fim = self.topologia['ramos'][nome_ramo]
            print(f"   Vazão em '{nome_ramo}' ({no_inicio} -> {no_fim}): {q_Ls:.8f} L/s")

    # Adiciona a função de chave de ordenação como um método da classe
    def _sort_key_ramo(self, ramo_name):
        match = re.search(r'(\d+)', ramo_name)
        return int(match.group(1)) if match else ramo_name    


In [26]:
df_pressoes = pd.read_excel('Dados CoW.xlsx', sheet_name = 'Pressões')
pressoes = dict(zip(df_pressoes['Nó'],  df_pressoes['Pressão (Pa)']))

df_geometrias = pd.read_excel('Dados CoW.xlsx', sheet_name = 'Geometrias')
geometrias = {row['Ramo']: (row['Comprimento 1 [m]'], row['Diâmetro 1 [m]'])
              for index, row in df_geometrias.iterrows()}

df_topologia = pd.read_excel("Dados CoW.xlsx", sheet_name = 'Topologia')
ramos = {row['Ramo']: (row['Inicio'], row['Fim'])
         for index, row in df_topologia.iterrows()}

df_nos_internos = pd.read_excel("Dados CoW.xlsx", sheet_name = 'Nós internos')
nos_internos = df_nos_internos['Nó'].tolist()

topologia = {'ramos': ramos, 'nos_internos': nos_internos}

teste = SistemaRedeArterial(
    P_contorno=pressoes,
    geometrias_ramos=geometrias,
    topologia=topologia
)

teste.exibir_equacoes()
NT = teste.resolver()
teste.exibir_resultados()


      Equações do Sistema a ser Resolvido

--- Equações de Perda de Carga (ΔP = R * Q) ---
  PPe - PA = Rramo1 * Qramo1
  PG - PH = Rramo10 * Qramo10
  PH - PI = Rramo11 * Qramo11
  PI - PJ = Rramo12 * Qramo12
  PJ - PK = Rramo13 * Qramo13
  PK - PPs = Rramo14 * Qramo14
  PK - PPs = Rramo15 * Qramo15
  PJ - PE = Rramo16 * Qramo16
  PB - PL = Rramo17 * Qramo17
  PL - PPs = Rramo18 * Qramo18
  PL - PPs = Rramo19 * Qramo19
  PA - PB = Rramo2 * Qramo2
  PG - PM = Rramo20 * Qramo20
  PM - PPs = Rramo21 * Qramo21
  PM - PPs = Rramo22 * Qramo22
  PC - PN = Rramo23 * Qramo23
  PN - PPs = Rramo24 * Qramo24
  PN - PPs = Rramo25 * Qramo25
  PO - PPs = Rramo26 * Qramo26
  PH - PO = Rramo27 * Qramo27
  PO - PPs = Rramo28 * Qramo28
  PI - PQ = Rramo29 * Qramo29
  PB - PC = Rramo3 * Qramo3
  PQ - PT = Rramo30 * Qramo30
  PQ - PU = Rramo31 * Qramo31
  PU - PPs = Rramo32 * Qramo32
  PU - PPs = Rramo33 * Qramo33
  PT - PPs = Rramo34 * Qramo34
  PT - PPs = Rramo35 * Qramo35
  PD - PP = Rramo36 * Qramo36

In [27]:
ramos_data = []

#Organizando em em forma crescente os ramos
def sort_key_ramo_excel(ramo_name): 
    match = re.search(r'(\d+)', ramo_name)
    return int(match.group(1)) if match else ramo_name 
lista_ordenada = sorted(teste.vasos, key=sort_key_ramo_excel)
nos_data = []
all_nodes_pressure = {**teste.P_contorno, **teste.pressoes_calculadas}

for ramo in lista_ordenada:
    vazao = teste.vazoes_calculadas_Ls[ramo]
    resistencia = teste.resistencias[ramo]
    no_inicio, no_fim = teste.topologia['ramos'][ramo]
    topologia_str = f"{no_inicio} -> {no_fim}"
    ramos_data.append([ramo, vazao, resistencia, topologia_str])

for node in sorted(all_nodes_pressure.keys()):
    pressure = all_nodes_pressure[node]
    nos_data.append([node, pressure])

df_ramos = pd.DataFrame(ramos_data, columns=['Ramos', 'Vazões (L/s)', 'Resistências Hidráulicas (Pa.s/m³)', 'Topologia'])

df_nos = pd.DataFrame(nos_data, columns=['Nó', 'Pressão nos Nós (Pa)'])



In [28]:
df_ramos.to_excel('resultados_Ramos.xlsx', index=False)
df_nos.to_excel('resultados_Nos.xlsx', index=False)

In [29]:
Ramos = pd.read_excel('C:\\Users\\Gabriel\\OneDrive - Unesp\\TCC\\resultados_Ramos.xlsx')
Nos = pd.read_excel('C:\\Users\\Gabriel\\OneDrive - Unesp\\TCC\\resultados_Nos.xlsx')


In [34]:
Ramos['Vazões (L/s)'].sum()

np.float64(0.008819304922404379)

In [38]:
PCA = Ramos['Vazões (L/s)'].iloc[[1, 8, 16, 17, 18, 19, 20, 21]].sum()
PCoA = Ramos['Vazões (L/s)'].iloc[[2, 9]].sum()
ICA = Ramos['Vazões (L/s)'].iloc[[3, 22, 23, 25, 26, 10]].sum()
MCA = Ramos['Vazões (L/s)'].iloc[[35, 36, 37, 41, 40, 38, 39, 28, 29, 30, 32, 31, 33, 34]].sum()
ACA = Ramos['Vazões (L/s)'].iloc[[4, 11, 5, 6, 7, 12, 13, 14]].sum()
ACoA = Ramos['Vazões (L/s)'].iloc[[15]].sum()
OA = Ramos['Vazões (L/s)'].iloc[[24, 27]].sum()
BA = Ramos['Vazões (L/s)'].iloc[[0]].sum()




print(f"\nA soma das vazões dos ramos ACoA é: {ACoA:} L/s")
print(f"\nA soma das vazões dos ramos PCoA é: {PCoA:} L/s")
print(f"\nA soma das vazões dos ramos PCA é: {PCA:} L/s")
print(f"\nA soma das vazões dos ramos ICA é: {ICA:} L/s")
print(f"\nA soma das vazões dos ramos MCA é: {MCA:} L/s")
print(f"\nA soma das vazões dos ramos ACA é: {ACA:} L/s")
print(f"\nA soma das vazões dos ramos OA é: {OA:} L/s")
#print(f"\nA soma das vazões dos ramos Total é: {ACoA + OA + PCA + PCoA + ICA + MCA + ACA:} L/s")
print(f"\nA soma das vazões dos ramos BA é: {BA:} L/s")


A soma das vazões dos ramos ACoA é: 9.591740715182287e-06 L/s

A soma das vazões dos ramos PCoA é: 0.000830761956315401 L/s

A soma das vazões dos ramos PCA é: 0.004219888249913704 L/s

A soma das vazões dos ramos ICA é: 0.0014864676257559588 L/s

A soma das vazões dos ramos MCA é: 0.000133347244058546 L/s

A soma das vazões dos ramos ACA é: 7.225526891313421e-05 L/s

A soma das vazões dos ramos OA é: 0.0001065221158842833 L/s

A soma das vazões dos ramos BA é: 0.001960470720848169 L/s


In [42]:
df_vasos = pd.DataFrame({ 'Artérias': ['ACoA', 'OA', 'PCA', 'PCoA', 'ICA', 'MCA', 'ACA', 'BA'], 
                          'Vazões (L/s)': [ACoA, OA, PCA, PCoA, ICA, MCA, ACA, BA] })

df_vasos

,Artérias,Vazões (L/s)
0,ACoA,0.000010
1,OA,0.000107
2,PCA,0.004220
3,PCoA,0.000831
4,ICA,0.001486
5,MCA,0.000133
6,ACA,0.000072
7,BA,0.001960


In [ ]:
# # =============================================================================
# # TESTE 1 = BIFURCAÇÃO.
# # =============================================================================


# pressões = {'P1': 12400, 'P2': 0, 'P3': 0}

# geometrias = {  'ramo1': (0.035, 0.004),  
#                 'ramo2': (0.016, 0.0026),    
#                 'ramo3': (0.014, 0.0024) }

                
# topologia = {'ramos': {
#         'ramo1': ('P1', 'A'), 'ramo2': ('A', 'P2'), 'ramo3': ('A', 'P3')},
#         'nos_internos': ['A']}

# teste = SistemaRedeArterial(
#     P_contorno=pressões,
#     geometrias_ramos=geometrias,
#     topologia=topologia
# )

# teste.exibir_equacoes()
# NT = teste.resolver()
# teste.exibir_resultados()

In [ ]:
# # =============================================================================
# # TESTE 2 = EXEMPLO INTERMEDIÁRIO DO CIRCULO DE WILLIS
# # =============================================================================

# pressões = { 'Pe': 12400 , 'Ps': 0 }

# geometrias = {
#     'ramo1': (0.10, 0.006), 'ramo2': (0.08, 0.003), 'ramo3': (0.08, 0.004),
#     'ramo4': (0.09, 0.0035), 'ramo5': (0.09, 0.0035), 'ramo6': (0.08, 0.005),
#     'ramo7': (0.10, 0.005)
# }
# topologia = {
#     'ramos': {
#         'ramo1': ('Pe', 'A'), 'ramo2': ('A', 'B'), 'ramo3': ('B', 'C'),
#         'ramo4': ('D', 'C'), 'ramo5': ('A', 'D'), 'ramo6': ('B', 'D'),
#         'ramo7': ('C', 'Ps')
#     },
#     'nos_internos': ['A', 'B', 'C', 'D']
# }


# teste = SistemaRedeArterial(
#     P_contorno=pressões,
#     geometrias_ramos=geometrias,
#     topologia=topologia
# )

# teste.exibir_equacoes()
# sucesso = teste.resolver()
# teste.exibir_resultados()


In [ ]:
# # =============================================================================
# # TESTE 3 = CIRCULO DE WILLIS COMPLETO
# # =============================================================================

# pressões = { 'Pe': 12400 , 'Ps': 0 }

# geometrias = {
#     'ramo1': (0.0304, 0.0017), 'ramo2': (0.0087, 0.0013), 'ramo3': (0.0146, 0.0007),
#     'ramo4': (0.0065, 0.0021), 'ramo5': (0.0217, 0.0012), 'ramo6': (0.0366, 0.001),
#     'ramo7': (0.0108, 0.0008), 'ramo8': (0.0108, 0.0008), 'ramo9': (0.0087, 0.0011), 
#     'ramo10': (0.0149, 0.0008), 'ramo11': (0.0062, 0.0023), 
#     'ramo12': (0.0189, 0.0012), 'ramo13': (0.0359, 0.001), 
#     'ramo14': (0.0119, 0.0008), 'ramo15': (0.0119, 0.0008), 
#     'ramo16': (0.0031, 0.0014), 'ramo17': (0.0257, 0.0011), 
#     'ramo18': (0.0146, 0.0008), 'ramo19': (0.0146, 0.0008), 
#     'ramo20': (0.028, 0.0011), 'ramo21': (0.0135, 0.0008), 
#     'ramo22': (0.0135, 0.0008), 'ramo23': (0.0088, 0.002), 
#     'ramo24': (0.0526, 0.0024), 'ramo25': (0.0082, 0.0009), 
#     'ramo26': (0.0548, 0.0024), 'ramo27': (0.0091, 0.0021), 
#     'ramo28': (0.0057, 0.0009), 'ramo29': (0.0251, 0.0017), 
#     'ramo30': (0.0261, 0.001), 'ramo31': (0.0261, 0.001), 
#     'ramo32': (0.0148, 0.0007), 'ramo33': (0.0148, 0.0007), 
#     'ramo34': (0.0148, 0.0007), 'ramo35': (0.0148, 0.0007), 
#     'ramo36': (0.0227, 0.0016), 'ramo37': (0.0293, 0.001),
#     'ramo38': (0.0293, 0.001), 'ramo39': (0.0144, 0.0007), 
#     'ramo40': (0.0144, 0.0007), 'ramo41': (0.0144, 0.0007), 'ramo42': (0.0144, 0.0007) 
# }

# topologia = {
#     'ramos': {
#         'ramo1': ('Pe', 'A'), 'ramo2': ('A', 'B'), 'ramo3': ('B', 'C'),
#         'ramo4': ('C', 'D'), 'ramo5': ('D', 'E'), 'ramo6': ('E', 'F'),
#         'ramo7': ('F', 'Ps'), 'ramo8':('F', 'Ps'),

#         'ramo9': ('A', 'G'), 'ramo10': ('G', 'H'), 'ramo11': ('H', 'I'),
#         'ramo12': ('I', 'J'), 'ramo13': ('J', 'K'), 'ramo14': ('K', 'Ps'),
#         'ramo15': ('K', 'Ps'),

#         'ramo16': ('E', 'J'),
        
#         'ramo17': ('B', 'L'), 'ramo18': ('L', 'Ps'), 'ramo19': ('L', 'Ps'),

#         'ramo20': ('G', 'M'), 'ramo21': ('M', 'Ps'), 'ramo22': ('M', 'Ps'),

#         'ramo23': ('N', 'C'), 'ramo24': ('Ps', 'N'), 'ramo25': ('N', 'Ps'),

#         'ramo26': ('Ps', 'O'), 'ramo27': ('O', 'H'), 'ramo28': ('O', 'Ps'),

#         'ramo29': ('I', 'Q'), 'ramo30': ('Q', 'T'), 'ramo31': ('Q', 'U'),
#         'ramo32': ('U', 'Ps'), 'ramo33': ('U', 'Ps'), 'ramo34': ('T', 'Ps'),
#         'ramo35': ('T', 'Ps'),

#         'ramo36': ('D', 'P'), 'ramo37': ('P', 'R'), 'ramo38': ('P', 'S'),
#         'ramo39': ('S', 'Ps'), 'ramo40': ('S', 'Ps'), 'ramo41': ('R', 'Ps'),
#         'ramo42': ('R', 'Ps') 
#     },
#     'nos_internos': ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K',
#                          'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U']
# }


# teste = SistemaRedeArterial(
#     P_contorno=pressões,
#     geometrias_ramos=geometrias,
#     topologia=topologia
# )

# teste.exibir_equacoes()
# sucesso = teste.resolver()
# teste.exibir_resultados()

In [ ]:
#Para salvar em excel, em planilhas diferentes

# output_file = 'Resultados_CoW.xlsx'
# with pd.ExcelWriter(output_file, engine='xlsxwriter') as writer:
#     df_ramos.to_excel(writer, sheet_name='Resultados Ramos', index=False)
#     df_nos.to_excel(writer, sheet_name='Resultados Nós', index=False)